<a href="https://colab.research.google.com/github/ChrizZhuang/memtransistor_NLP/blob/main/MLP_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import time
from datetime import datetime
import pandas as pd

TensorFlow 1.x selected.


In [5]:
# imports data from single column CSV file with possible current/conductance states
# return numpy array of approximate states possible using this hardware
# this import method is not generalized, but fine-tuned to Vinod's devices
def import_data_from_csv(filename):
  # import data
  imported_device_states = np.genfromtxt(filename, delimiter=',')[1:]

  # since data is in ~1 nA, assume maximum precision is ~1 pA
  # this will make some states redundant
  imported_device_states = np.unique(np.round(np.sort(imported_device_states), decimals=3))

  # calculate device states possible
  device_states = np.array([])
  for i, value in enumerate(imported_device_states):
      if i+1 > len(imported_device_states):
          break
      temp_ls = value - imported_device_states
      device_states = np.append(device_states, temp_ls)


  # normalize to -1 to 1
  device_states = np.unique(np.sort(device_states))
  device_states = device_states / np.abs(device_states).max()

  # given the large number of states, we can assume some states are almost equivalent
  # moreover, once the number of states is > 100, the discreteness doesnt matter
  # for simplicity in the simulations, we will simply  round to 2 digits of the calculated states
  device_states = np.round(device_states, decimals = 2)
  device_states = np.unique(np.sort(device_states))
  
  return device_states

In [6]:
###################### USER DEFINED PARAMETERS FOR SIMULATION
# number of epochs to test
NUM_EPOCHS = 100
# parameter set by user that gives all the possible normalized weight states
# assumes (1) discrete number of states that are normalized 
#         (2) states are set by two synaptic devices such that weight = weight_p - weight_m
#         (3) because of (2), weights can vary from [-1,1]
# user input = a 1D numpy array with values from [-1, 1]
DEVICE_STATES = import_data_from_csv('learning_curve_vinod.csv')
# print(len(DEVICE_STATES))

# parameters for simulating read noise
# user input = read noise mean and standard dev assuming a normal noise function
READ_NOISE_MEAN = 0
READ_NOISE_STDDEV = 0.1

# parameter for simulating device-to-device variation
# user input =  standard deviation of conductances
DEVICE_VARIATION_STDDEV = 0.1

# parameter for simulating devices that get stuck on Gmax or Gmin states from the start
# user input = probability for a device to get stuck
DEVICE_STUCK_ON_PROB = 0.1
DEVICE_STUCK_OFF_PROB = 0.1

###################### SIM PARAMETERS
n_inputs = 28*28  # MNIST
n_hidden1 = 300 # neurons in 1st hidden layers
n_outputs = 10 # neurons in output layer
learning_rate = 0.1#0.01 # grad descent
initializer_stddev = 0.2 # standar deviation of initialized random weights
n_epochs = 10 # number of epochs to test
batch_size = 50 # batch size before tuning weights in grad descent

g_min_value = np.min(np.abs(DEVICE_STATES))
g_max_value = np.max(np.abs(DEVICE_STATES))

# define weight update ops
var_stuck_mat = [False, False, False, False] # if each device is stucked
weights = [0,0,0,0]
new_weights = [0,0,0,0]
weight_update_op = [0,0,0,0]

In [7]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# will create matrix to simulate device-to-device variation by creating clipping the weights
# will also simulate devices being stuck-on-open and stuck-on-close 
def initialize_variation_stuck_mat(shape):

    # VARIATION
    wp_max = np.ones(shape=shape) - np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))
    wp_min = np.zeros(shape=shape) + np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))

    wm_max = np.ones(shape=shape) - np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))
    wm_min = np.zeros(shape=shape) + np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))

    # STUCK
    stuck_prob = [DEVICE_STUCK_OFF_PROB, 1 - DEVICE_STUCK_ON_PROB - DEVICE_STUCK_OFF_PROB, DEVICE_STUCK_ON_PROB]
    w_p_stuck = np.random.choice([-1, 0, 1], size=shape, p=stuck_prob) # choose from [-1, 0, 1] with probability of [DEVICE_STUCK_OFF_PROB, 1 - DEVICE_STUCK_ON_PROB - DEVICE_STUCK_OFF_PROB, DEVICE_STUCK_ON_PROB]
    w_m_stuck = np.random.choice([-1, 0, 1], size=shape, p=stuck_prob)

    # if device is stuck OFF
    wp_max = wp_max + (w_p_stuck == -1) * (wp_min - wp_max)
    wm_max = wm_max + (w_m_stuck == -1) * (wm_min - wm_max)

    # if device is stuck ON
    wp_min = wp_min + (w_p_stuck == 1) * (wp_max - wp_min)
    wm_min = wm_min + (w_m_stuck == 1) * (wm_max - wm_min)


    # PUTTING TOGETHER CLIPPING MATRIX
    lower_lim = np.clip(wp_min - wm_max, -g_max_value, -g_min_value)
    upper_lim = np.clip(wp_max - wm_min, g_min_value, g_max_value)

    # 2 matrices with shape = shape that contains the variation of weights
    return [lower_lim, upper_lim]

In [8]:
shape = np.array(np.array(weights).shape)
mat = initialize_variation_stuck_mat(shape)
#mat

In [9]:
# weight update with a discrete number of states and (optional) add read noise
def discrete_weight_update(value, read_noise_mean=0, read_noise_stddev=0):
    if read_noise_stddev != 0:
        value += np.random.normal(read_noise_mean, read_noise_stddev)
    absolute_difference_function = lambda list_value : abs(list_value - value)
    return min(DEVICE_STATES, key=absolute_difference_function)
v_discrete_weight_update = np.vectorize(discrete_weight_update)

In [10]:
new_weights_mat = v_discrete_weight_update(weights, read_noise_mean = READ_NOISE_MEAN,
              read_noise_stddev = READ_NOISE_STDDEV)
new_weights_mat

array([ 0.11,  0.08, -0.03,  0.02])

In [11]:
def simulate_hardware_weight_update(weights_mat, var_stuck_mat):

    # initialize variation and stuck matrix if not initialized
    if type(var_stuck_mat) is not np.ndarray:
        var_stuck_mat = initialize_variation_stuck_mat(weights_mat.shape)

    # simulate weight variation and stuck on open/close
    weights_mat = weights_mat.clip(var_stuck_mat[0], var_stuck_mat[1])

    # simulate discrete states
    weights_mat = v_discrete_weight_update(weights_mat, read_noise_mean = READ_NOISE_MEAN,
                                read_noise_stddev = READ_NOISE_STDDEV)

    return weights_mat

In [ ]:
simulate_hardware_weight_update(np.array(weights), var_stuck_mat)

array([-0.0087554 , -0.0087554 ,  0.12154736,  0.10809185])

In [ ]:
simulate_hardware_weight_update(new_weights_mat, var_stuck_mat)

array([ 0.15373901, -0.04164841, -0.0087554 , -0.0087554 ])

In [12]:
# reset default tf graph before running sim
reset_graph()
# get MNIST data, format
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
#X_valid, X_train = X_train[:5000], X_train[5000:]
#y_valid, y_train = y_train[:5000], y_train[5000:]


# define input and output placeholder variables
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") # input
y = tf.placeholder(tf.int32, shape=(None), name="y") # output


#with tf.name_scope("dnn"):
initiliazer = tf.truncated_normal_initializer(stddev = initializer_stddev)
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu, 
                          kernel_initializer=initiliazer, bias_initializer=initiliazer)
logits = tf.layers.dense(hidden1, n_outputs, name="outputs",
                          kernel_initializer=initiliazer, bias_initializer=initiliazer)
y_proba = tf.nn.softmax(logits)

tf.trainable_variables()

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'outputs/kernel:0' shape=(300, 10) dtype=float32_ref>,
 <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]

In [13]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [14]:
weight_layers = ["hidden1/kernel:0", "hidden1/bias:0", "outputs/kernel:0", "outputs/bias:0"]
for i, name in enumerate(weight_layers):
  #print("i = " + str(i))
  #print("name = " + str(name))
  weights[i] = [v for v in tf.trainable_variables() if v.name == name][0]
  #print("weights = " + str(weights))
  new_weights[i] = tf.placeholder(tf.float32, name="new_weights"+name.replace("/","-").replace(":","-"))
  print("new_weights = " + str(new_weights))
  weight_update_op[i] = tf.assign(weights[i], new_weights[i])
  print("weight_update_op = " + str(weight_update_op))

new_weights = [<tf.Tensor 'new_weightshidden1-kernel-0:0' shape=<unknown> dtype=float32>, 0, 0, 0]
weight_update_op = [<tf.Tensor 'Assign:0' shape=(784, 300) dtype=float32_ref>, 0, 0, 0]
new_weights = [<tf.Tensor 'new_weightshidden1-kernel-0:0' shape=<unknown> dtype=float32>, <tf.Tensor 'new_weightshidden1-bias-0:0' shape=<unknown> dtype=float32>, 0, 0]
weight_update_op = [<tf.Tensor 'Assign:0' shape=(784, 300) dtype=float32_ref>, <tf.Tensor 'Assign_1:0' shape=(300,) dtype=float32_ref>, 0, 0]
new_weights = [<tf.Tensor 'new_weightshidden1-kernel-0:0' shape=<unknown> dtype=float32>, <tf.Tensor 'new_weightshidden1-bias-0:0' shape=<unknown> dtype=float32>, <tf.Tensor 'new_weightsoutputs-kernel-0:0' shape=<unknown> dtype=float32>, 0]
weight_update_op = [<tf.Tensor 'Assign:0' shape=(784, 300) dtype=float32_ref>, <tf.Tensor 'Assign_1:0' shape=(300,) dtype=float32_ref>, <tf.Tensor 'Assign_2:0' shape=(300, 10) dtype=float32_ref>, 0]
new_weights = [<tf.Tensor 'new_weightshidden1-kernel-0:0' shap

In [15]:
weights

[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>,
 <tf.Variable 'outputs/kernel:0' shape=(300, 10) dtype=float32_ref>,
 <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]

In [16]:
new_weights

[<tf.Tensor 'new_weightshidden1-kernel-0:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'new_weightshidden1-bias-0:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'new_weightsoutputs-kernel-0:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'new_weightsoutputs-bias-0:0' shape=<unknown> dtype=float32>]

In [17]:
weight_update_op

[<tf.Tensor 'Assign:0' shape=(784, 300) dtype=float32_ref>,
 <tf.Tensor 'Assign_1:0' shape=(300,) dtype=float32_ref>,
 <tf.Tensor 'Assign_2:0' shape=(300, 10) dtype=float32_ref>,
 <tf.Tensor 'Assign_3:0' shape=(10,) dtype=float32_ref>]

In [18]:
epoch_ls = []
recognition_rate_ls = []
start_time = time.time()
start_datetime  = datetime.utcnow().strftime("%Y%m%d%H%M%S")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

weights_before = False
weights_after = False

In [19]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [25]:
HARDWARE_SIMULATION = True
with tf.Session() as sess:
  init.run()

  for epoch in range(n_epochs):
      for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
          sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

      if HARDWARE_SIMULATION:
          ###### WEIGHT UPDATE  
          # simulate hardware by updating weights
          # includes discrete number of weight states

          # the code below only updates hidden1/kernel:0 weights

          # get weights
          #weights_temp = [v for v in tf.trainable_variables() if v.name == "hidden1/bias:0"][0]
          #weights_before = weights_temp.eval(session=sess)


          # simulate hardware weight update
          #new_weights_mat = simulate_hardware_weight_update(weights_mat, var_stuck_mat)

          # update weights to hardware simulated weights

          for i, weight in enumerate(weights):
              print("i = " + str(i))
              weight = weight.eval(session=sess)
              print(weight.shape)
              new_weights_mat = simulate_hardware_weight_update(weight, var_stuck_mat[i])
              print(new_weights_mat.shape)
              weight_update_op[i].eval(feed_dict={new_weights[i]: new_weights_mat})


          ###### end of WEIGHT UPDATE



      # test accuracy
      acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
      acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
      print(epoch, "Batch accuracy:", acc_batch, "Test accuracy:", acc_valid)
      
      # save to list
      epoch_ls.append(epoch)
      recognition_rate_ls.append(acc_valid)

i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
0 Batch accuracy: 0.76 Test accuracy: 0.7274
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
1 Batch accuracy: 0.84 Test accuracy: 0.7122
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
2 Batch accuracy: 0.54 Test accuracy: 0.6534
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
3 Batch accuracy: 0.8 Test accuracy: 0.7726
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
4 Batch accuracy: 0.78 Test accuracy: 0.6961
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
5 Batch accuracy: 0.64 Test accuracy: 0.6928
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(300,)
i = 2
(300, 10)
(300, 10)
i = 3
(10,)
(10,)
6 Batch accuracy: 0.6 Test accuracy: 0.6384
i = 0
(784, 300)
(784, 300)
i = 1
(300,)
(3

In [ ]:
def run_MLP_simulation(save_results_input = False,
                       num_epochs_input=50, 
                       hardware_simulation_input=False, 
                       device_states_input=False,
                       read_noise_mean_input=0,
                       read_noise_stddev_input=0,
                       device_variation_stddev_input=0,
                       device_stuck_on_prob_input=0,
                       device_stuck_off_prob_input=0):
    
    
    
    ###################### USER DEFINED PARAMETERS FOR SIMULATION
    # whether or not to save results
    SAVE_RESULTS = save_results_input
    
    # number of epochs to test
    NUM_EPOCHS = num_epochs_input

    # flag which determines whether this is a hardware simulation or purely software
    HARDWARE_SIMULATION = hardware_simulation_input

    # parameter set by user that gives all the possible normalized weight states
    # assumes (1) discrete number of states that are normalized 
    #         (2) states are set by two synaptic devices such that weight = weight_p - weight_m
    #         (3) because of (2), weights can vary from [-1,1]
    # user input = a 1D numpy array with values from [-1, 1]
    DEVICE_STATES = device_states_input


    # parameters for simulating read noise
    # user input = read noise mean and standard dev assuming a normal noise function
    READ_NOISE_MEAN = read_noise_mean_input
    READ_NOISE_STDDEV = read_noise_stddev_input

    # parameter for simulating device-to-device variation
    # user input =  standard deviation of conductances
    DEVICE_VARIATION_STDDEV = device_variation_stddev_input

    # parameter for simulating devices that get stuck on Gmax or Gmin states from the start
    # user input = probability for a device to get stuck
    DEVICE_STUCK_ON_PROB = device_stuck_on_prob_input
    DEVICE_STUCK_OFF_PROB = device_stuck_off_prob_input

    
    
    ###################### SIM PARAMETERS
    n_inputs = 28*28  # MNIST
    n_hidden1 = 300 # neurons in 1st hidden layers
    n_outputs = 10 # neurons in output layer
    learning_rate = 0.1#0.01 # grad descent
    initializer_stddev = 0.2 # standar deviation of initialized random weights
    n_epochs = NUM_EPOCHS # number of epochs to test
    batch_size = 50 # batch size before tuning weights in grad descent

    g_min_value = np.min(np.abs(DEVICE_STATES))
    g_max_value = np.max(np.abs(DEVICE_STATES))
    
    
    
    
    ###################### FUNCTIONS

    # to make this notebook's output stable across runs
    def reset_graph(seed=42):
        tf.reset_default_graph()
        tf.set_random_seed(seed)
        np.random.seed(seed)

    # will create matrix to simulate device-to-device variation by creating clipping the weights
    # will also simulate devices being stuck-on-open and stuck-on-close 
    def initialize_variation_stuck_mat(shape):

        # VARIATION
        wp_max = np.ones(shape=shape) - np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))
        wp_min = np.zeros(shape=shape) + np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))

        wm_max = np.ones(shape=shape) - np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))
        wm_min = np.zeros(shape=shape) + np.abs(np.random.normal(0, DEVICE_VARIATION_STDDEV, shape))

        # STUCK
        stuck_prob = [DEVICE_STUCK_OFF_PROB, 1 - DEVICE_STUCK_ON_PROB - DEVICE_STUCK_OFF_PROB, DEVICE_STUCK_ON_PROB]
        w_p_stuck = np.random.choice([-1, 0, 1], size=shape, p=stuck_prob) # choose from [-1, 0, 1] with probability of [DEVICE_STUCK_OFF_PROB, 1 - DEVICE_STUCK_ON_PROB - DEVICE_STUCK_OFF_PROB, DEVICE_STUCK_ON_PROB]
        w_m_stuck = np.random.choice([-1, 0, 1], size=shape, p=stuck_prob)

        # if device is stuck OFF
        wp_max = wp_max + (w_p_stuck == -1) * (wp_min - wp_max)
        wm_max = wm_max + (w_m_stuck == -1) * (wm_min - wm_max)

        # if device is stuck ON
        wp_min = wp_min + (w_p_stuck == 1) * (wp_max - wp_min)
        wm_min = wm_min + (w_m_stuck == 1) * (wm_max - wm_min)


        # PUTTING TOGETHER CLIPPING MATRIX
        lower_lim = np.clip(wp_min - wm_max, -g_max_value, -g_min_value)
        upper_lim = np.clip(wp_max - wm_min, g_min_value, g_max_value)

        return [lower_lim, upper_lim]


    # weight update with a discrete number of states and (optional) add read noise
    def discrete_weight_update(value, read_noise_mean=0, read_noise_stddev=0):
        if read_noise_stddev != 0:
            value += np.random.normal(read_noise_mean, read_noise_stddev)
        absolute_difference_function = lambda list_value : abs(list_value - value)
        return min(DEVICE_STATES, key=absolute_difference_function)
    v_discrete_weight_update = np.vectorize(discrete_weight_update)



    # function puts together all the parts
    # 1. Device variation
    # 2. Stuck-on/off 
    # 3. Discrete number of weight states
    # Input = software weights matrix, Output = hardware weights matrix
    def simulate_hardware_weight_update(weights_mat, var_stuck_mat):

        # initialize variation and stuck matrix if not initialized
        if type(var_stuck_mat) is not np.ndarray:
            var_stuck_mat = initialize_variation_stuck_mat(weights_mat.shape)

        # simulate weight variation and stuck on open/close
        weights_mat = weights_mat.clip(var_stuck_mat[0], var_stuck_mat[1])

        # simulate discrete states
        weights_mat = v_discrete_weight_update(weights_mat, read_noise_mean = READ_NOISE_MEAN,
                                                 read_noise_stddev = READ_NOISE_STDDEV)

        return weights_mat
    
    
    ###################### MLP SIM SETUP
    # reset default tf graph before running sim
    reset_graph()

    # get MNIST data, format
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
    X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    #X_valid, X_train = X_train[:5000], X_train[5000:]
    #y_valid, y_train = y_train[:5000], y_train[5000:]


    # define input and output placeholder variables
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") # input
    y = tf.placeholder(tf.int32, shape=(None), name="y") # output

    # define NN layers
    #with tf.name_scope("dnn"):
    initiliazer = tf.truncated_normal_initializer(stddev = initializer_stddev)
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu, 
                              kernel_initializer=initiliazer, bias_initializer=initiliazer)
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs",
                              kernel_initializer=initiliazer, bias_initializer=initiliazer)
    y_proba = tf.nn.softmax(logits)

    # define loss
    #with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

    # define training
    #with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

    # define recognition rate eval op
    #with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    # define weight update ops
    var_stuck_mat = [False, False, False, False]
    weights = [0,0,0,0]
    new_weights = [0,0,0,0]
    weight_update_op = [0,0,0,0]
    with tf.name_scope("weight_update"):
        weight_layers = ["hidden1/kernel:0", "hidden1/bias:0", "outputs/kernel:0", "outputs/bias:0"]
        for i, name in enumerate(weight_layers):
            weights[i] = [v for v in tf.trainable_variables() if v.name == name][0]
            new_weights[i] = tf.placeholder(tf.float32, name="new_weights"+name.replace("/","-").replace(":","-"))
            weight_update_op[i] = tf.assign(weights[i], new_weights[i])
    # len(weights) = 4
    # weights[i].shape = (784, 300), (300,), (300, 10), (10,)    
    
    
    ###################### MLP SIM RUN
    epoch_ls = []
    recognition_rate_ls = []
    start_time = time.time()
    start_datetime  = datetime.utcnow().strftime("%Y%m%d%H%M%S")

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    weights_before = False
    weights_after = False

    def shuffle_batch(X, y, batch_size):
        rnd_idx = np.random.permutation(len(X))
        n_batches = len(X) // batch_size
        for batch_idx in np.array_split(rnd_idx, n_batches):
            X_batch, y_batch = X[batch_idx], y[batch_idx]
            yield X_batch, y_batch

    with tf.Session() as sess:
        init.run()

        for epoch in range(n_epochs):
            for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            if HARDWARE_SIMULATION:
                ###### WEIGHT UPDATE  
                # simulate hardware by updating weights
                # includes discrete number of weight states

                # the code below only updates hidden1/kernel:0 weights

                # get weights
                #weights_temp = [v for v in tf.trainable_variables() if v.name == "hidden1/bias:0"][0]
                #weights_before = weights_temp.eval(session=sess)


                # simulate hardware weight update
                #new_weights_mat = simulate_hardware_weight_update(weights_mat, var_stuck_mat)

                # update weights to hardware simulated weights

                for i, weight in enumerate(weights):

                    weight = weight.eval(session=sess)
                    new_weights_mat = simulate_hardware_weight_update(weight, var_stuck_mat[i])
                    weight_update_op[i].eval(feed_dict={new_weights[i]: new_weights_mat})


                ###### end of WEIGHT UPDATE



            # test accuracy
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Batch accuracy:", acc_batch, "Test accuracy:", acc_valid)
            
            # save to list
            epoch_ls.append(epoch)
            recognition_rate_ls.append(acc_valid)

            #weights_temp = [v for v in tf.trainable_variables() if v.name == "hidden1/bias:0"][0]
            #weights_after = weights_temp.eval(session=sess)    


        # save results
        if SAVE_RESULTS:
            save_path = "./MLP_sim_results/MLP_sim_" + start_datetime + "/model_" + start_datetime
            save_path = saver.save(sess, save_path + ".ckpt")
    
    # print out duration
    print("--- %0.2f seconds ---" % (time.time() - start_time))
    
    # save all results, including testing parameters and data
    if SAVE_RESULTS:
        save_path = "./MLP_sim_results/MLP_sim_" + start_datetime + "/model_" + start_datetime

        data_df = pd.DataFrame({"Epoch": epoch_ls, "Recognition Rate": recognition_rate_ls})
        data_df.to_csv(save_path + "_data.csv", index=False,)

        with open(save_path + "_meta.txt", "w") as text_file:
            print("---------- User input parameters -------------", file=text_file)
            print("Simulation start time: {}".format(start_datetime), file=text_file)
            print("Duration: {}".format(time.time() - start_time), file=text_file)
            print("Epochs: {}".format(num_epochs_input), file=text_file)
            print("Hardware simulation?: {}".format(hardware_simulation_input), file=text_file)
            print("Read noise - mean: {}".format(read_noise_mean_input), file=text_file)
            print("Read noise - standard deviation: {}".format(read_noise_stddev_input), file=text_file)
            print("Device variation - standard deviation: {}".format(device_variation_stddev_input), file=text_file)
            print("Device stuck on probability: {}".format(device_stuck_on_prob_input), file=text_file)
            print("Device stuck off probability: {}".format(device_stuck_off_prob_input), file=text_file)
            print("Device states used: {}".format(device_states_input), file=text_file)

            print("---------- Simulation parameters -------------", file=text_file)
            print("Number of inputs: {}".format(n_inputs), file=text_file)
            print("Layers: {}".format(weight_layers), file=text_file)
            print("Hidden1 # of neurons: {}".format(n_hidden1), file=text_file)
            print("Outputs # of neurons: {}".format(n_outputs), file=text_file)
            print("Learning rate: {}".format(learning_rate), file=text_file)
            print("Initializer standad dev: {}".format(initializer_stddev), file=text_file)
            print("Batch size: {}".format(batch_size), file=text_file)


# imports data from single column CSV file with possible current/conductance states
# return numpy array of approximate states possible using this hardware
# this import method is not generalized, but fine-tuned to Vinod's devices
def import_data_from_csv(filename):
    # import data
    imported_device_states = np.genfromtxt(filename, delimiter=',')[1:]

    # since data is in ~1 nA, assume maximum precision is ~1 pA
    # this will make some states redundant
    imported_device_states = np.unique(np.round(np.sort(imported_device_states), decimals=3))
    #print(len(imported_device_states))
    # calculate device states possible
    device_states = np.array([])
    for i, value in enumerate(imported_device_states):
      #print(i)
      temp_ls = value - imported_device_states
      device_states = np.append(device_states, temp_ls)

    #print(len(device_states))
    # normalize to -1 to 1
    device_states = np.unique(np.sort(device_states))
    device_states = device_states / np.abs(device_states).max()
    #print(len(device_states))
    # given the large number of states, we can assume some states are almost equivalent
    # moreover, once the number of states is > 100, the discreteness doesnt matter
    # for simplicity in the simulations, we will simply  round to 2 digits of the calculated states
    device_states = np.round(device_states, decimals = 2)
    device_states = np.unique(np.sort(device_states))
    #print(len(device_states))
    return device_states

In [ ]:
for i in range(1):
  imported_device_states = import_data_from_csv(filename='learning_curve_vinod.csv')
  run_MLP_simulation(num_epochs_input = 2, 
            hardware_simulation_input = True, 
            device_states_input = imported_device_states,
            read_noise_mean_input = 0.0,
            read_noise_stddev_input = 0.0,
            device_variation_stddev_input = 0.0,
            device_stuck_on_prob_input = 0.0,
            device_stuck_off_prob_input = 0.0,
            save_results_input = False)

(784, 300)
(300,)
(300, 10)
(10,)
0 Batch accuracy: 0.94 Test accuracy: 0.9442
1 Batch accuracy: 0.96 Test accuracy: 0.9567
--- 55.24 seconds ---


In [ ]:
# run hardware sim 4 times
for i in range(4):
    imported_device_states = import_data_from_csv(filename='learning_curve_vinod.csv')
    run_MLP_simulation(num_epochs_input = 100, 
                       hardware_simulation_input = True, 
                       device_states_input = imported_device_states,
                       read_noise_mean_input = 0.0,
                       read_noise_stddev_input = 0.1,
                       device_variation_stddev_input = 0.0,
                       device_stuck_on_prob_input = 0.00,
                       device_stuck_off_prob_input = 0.00,
                       save_results_input = True)

0 Batch accuracy: 0.72 Test accuracy: 0.6556
1 Batch accuracy: 0.6 Test accuracy: 0.5914
2 Batch accuracy: 0.7 Test accuracy: 0.7075
3 Batch accuracy: 0.5 Test accuracy: 0.5555
4 Batch accuracy: 0.82 Test accuracy: 0.7896
5 Batch accuracy: 0.76 Test accuracy: 0.7968
6 Batch accuracy: 0.8 Test accuracy: 0.776
7 Batch accuracy: 0.82 Test accuracy: 0.7581
8 Batch accuracy: 0.9 Test accuracy: 0.8142
9 Batch accuracy: 0.6 Test accuracy: 0.6915
10 Batch accuracy: 0.78 Test accuracy: 0.8267
11 Batch accuracy: 0.9 Test accuracy: 0.8334
12 Batch accuracy: 0.84 Test accuracy: 0.8398
13 Batch accuracy: 0.82 Test accuracy: 0.8183
14 Batch accuracy: 0.8 Test accuracy: 0.843
15 Batch accuracy: 0.92 Test accuracy: 0.846
16 Batch accuracy: 0.78 Test accuracy: 0.826
17 Batch accuracy: 0.82 Test accuracy: 0.8299
18 Batch accuracy: 0.84 Test accuracy: 0.8476
19 Batch accuracy: 0.86 Test accuracy: 0.8514
20 Batch accuracy: 0.84 Test accuracy: 0.8119
21 Batch accuracy: 0.88 Test accuracy: 0.829
22 Batch ac

80 Batch accuracy: 0.9 Test accuracy: 0.8741
81 Batch accuracy: 0.8 Test accuracy: 0.8845
82 Batch accuracy: 0.94 Test accuracy: 0.8958
83 Batch accuracy: 0.88 Test accuracy: 0.9004
84 Batch accuracy: 0.9 Test accuracy: 0.8872
85 Batch accuracy: 0.82 Test accuracy: 0.8762
86 Batch accuracy: 0.9 Test accuracy: 0.885
87 Batch accuracy: 0.84 Test accuracy: 0.8654
88 Batch accuracy: 0.94 Test accuracy: 0.8849
89 Batch accuracy: 0.86 Test accuracy: 0.8693
90 Batch accuracy: 0.94 Test accuracy: 0.8902
91 Batch accuracy: 0.9 Test accuracy: 0.9053
92 Batch accuracy: 0.84 Test accuracy: 0.8824
93 Batch accuracy: 0.88 Test accuracy: 0.8821
94 Batch accuracy: 0.86 Test accuracy: 0.875
95 Batch accuracy: 0.98 Test accuracy: 0.8869
96 Batch accuracy: 0.92 Test accuracy: 0.8769
97 Batch accuracy: 0.82 Test accuracy: 0.8863
98 Batch accuracy: 0.88 Test accuracy: 0.8889
99 Batch accuracy: 0.94 Test accuracy: 0.8792
--- 3072.75 seconds ---
0 Batch accuracy: 0.72 Test accuracy: 0.6556
1 Batch accuracy: 

60 Batch accuracy: 0.88 Test accuracy: 0.8763
61 Batch accuracy: 0.88 Test accuracy: 0.8825
62 Batch accuracy: 0.94 Test accuracy: 0.8704
63 Batch accuracy: 0.88 Test accuracy: 0.8854
64 Batch accuracy: 0.84 Test accuracy: 0.8656
65 Batch accuracy: 0.9 Test accuracy: 0.8631
66 Batch accuracy: 0.92 Test accuracy: 0.8702
67 Batch accuracy: 0.92 Test accuracy: 0.8812
68 Batch accuracy: 0.9 Test accuracy: 0.8789
69 Batch accuracy: 0.9 Test accuracy: 0.8849
70 Batch accuracy: 0.88 Test accuracy: 0.8887
71 Batch accuracy: 0.88 Test accuracy: 0.8795
72 Batch accuracy: 0.9 Test accuracy: 0.8821
73 Batch accuracy: 0.94 Test accuracy: 0.8834
74 Batch accuracy: 0.9 Test accuracy: 0.8834
75 Batch accuracy: 0.94 Test accuracy: 0.884
76 Batch accuracy: 0.92 Test accuracy: 0.8814
77 Batch accuracy: 0.9 Test accuracy: 0.8798
78 Batch accuracy: 0.92 Test accuracy: 0.8825
79 Batch accuracy: 0.92 Test accuracy: 0.886
80 Batch accuracy: 0.9 Test accuracy: 0.8741
81 Batch accuracy: 0.8 Test accuracy: 0.884

In [ ]:
# run hardware sim 4 times
for i in range(4):
    imported_device_states = import_data_from_csv(filename='learning_curve_vinod.csv')
    run_MLP_simulation(num_epochs_input = 100, 
                       hardware_simulation_input = True, 
                       device_states_input = imported_device_states,
                       read_noise_mean_input = 0.0,
                       read_noise_stddev_input = 0.1,
                       device_variation_stddev_input = 0.0,
                       device_stuck_on_prob_input = 0.00,
                       device_stuck_off_prob_input = 0.00,
                       save_results_input = True)

0 Batch accuracy: 0.84 Test accuracy: 0.761
1 Batch accuracy: 0.86 Test accuracy: 0.8079
2 Batch accuracy: 0.88 Test accuracy: 0.8493
3 Batch accuracy: 0.84 Test accuracy: 0.823
4 Batch accuracy: 0.92 Test accuracy: 0.9038
5 Batch accuracy: 0.86 Test accuracy: 0.9098
6 Batch accuracy: 0.96 Test accuracy: 0.9185
7 Batch accuracy: 0.92 Test accuracy: 0.8956
8 Batch accuracy: 0.94 Test accuracy: 0.9088
9 Batch accuracy: 0.86 Test accuracy: 0.8886
10 Batch accuracy: 0.92 Test accuracy: 0.9233
11 Batch accuracy: 0.94 Test accuracy: 0.919
12 Batch accuracy: 0.84 Test accuracy: 0.917
13 Batch accuracy: 0.96 Test accuracy: 0.9239
14 Batch accuracy: 0.9 Test accuracy: 0.9272
15 Batch accuracy: 0.94 Test accuracy: 0.9344
16 Batch accuracy: 0.94 Test accuracy: 0.9288
17 Batch accuracy: 0.96 Test accuracy: 0.9197
18 Batch accuracy: 0.9 Test accuracy: 0.9308
19 Batch accuracy: 0.88 Test accuracy: 0.9271
20 Batch accuracy: 0.96 Test accuracy: 0.9229
21 Batch accuracy: 0.92 Test accuracy: 0.9356
22 B

79 Batch accuracy: 0.98 Test accuracy: 0.9432
80 Batch accuracy: 0.96 Test accuracy: 0.9453
81 Batch accuracy: 0.98 Test accuracy: 0.9485
82 Batch accuracy: 0.96 Test accuracy: 0.9482
83 Batch accuracy: 0.96 Test accuracy: 0.9417
84 Batch accuracy: 0.96 Test accuracy: 0.9472
85 Batch accuracy: 0.94 Test accuracy: 0.9449
86 Batch accuracy: 0.98 Test accuracy: 0.9481
87 Batch accuracy: 0.98 Test accuracy: 0.9496
88 Batch accuracy: 0.96 Test accuracy: 0.9505
89 Batch accuracy: 0.94 Test accuracy: 0.9453
90 Batch accuracy: 0.98 Test accuracy: 0.9494
91 Batch accuracy: 1.0 Test accuracy: 0.952
92 Batch accuracy: 0.96 Test accuracy: 0.9466
93 Batch accuracy: 0.94 Test accuracy: 0.9496
94 Batch accuracy: 0.94 Test accuracy: 0.944
95 Batch accuracy: 1.0 Test accuracy: 0.9475
96 Batch accuracy: 0.96 Test accuracy: 0.9516
97 Batch accuracy: 0.96 Test accuracy: 0.9425
98 Batch accuracy: 0.96 Test accuracy: 0.9457
99 Batch accuracy: 0.94 Test accuracy: 0.9451
--- 3079.70 seconds ---
0 Batch accura

58 Batch accuracy: 1.0 Test accuracy: 0.9432
59 Batch accuracy: 0.96 Test accuracy: 0.9416
60 Batch accuracy: 0.9 Test accuracy: 0.9444
61 Batch accuracy: 0.98 Test accuracy: 0.9438
62 Batch accuracy: 0.94 Test accuracy: 0.9366
63 Batch accuracy: 0.96 Test accuracy: 0.9392
64 Batch accuracy: 0.98 Test accuracy: 0.9374
65 Batch accuracy: 0.98 Test accuracy: 0.947
66 Batch accuracy: 0.98 Test accuracy: 0.9456
67 Batch accuracy: 0.98 Test accuracy: 0.9416
68 Batch accuracy: 0.94 Test accuracy: 0.9412
69 Batch accuracy: 0.92 Test accuracy: 0.9447
70 Batch accuracy: 0.94 Test accuracy: 0.9425
71 Batch accuracy: 0.96 Test accuracy: 0.9431
72 Batch accuracy: 0.96 Test accuracy: 0.9421
73 Batch accuracy: 0.96 Test accuracy: 0.9363
74 Batch accuracy: 0.96 Test accuracy: 0.9467
75 Batch accuracy: 0.98 Test accuracy: 0.9457
76 Batch accuracy: 0.94 Test accuracy: 0.9445
77 Batch accuracy: 0.98 Test accuracy: 0.9473
78 Batch accuracy: 0.96 Test accuracy: 0.9428
79 Batch accuracy: 0.98 Test accuracy

In [ ]:
imported_device_states = import_data_from_csv(filename='learning_curve_vinod.csv')


run_MLP_simulation(num_epochs_input = 100, 
                   hardware_simulation_input = True, 
                   device_states_input = imported_device_states,
                   read_noise_mean_input = 0.0,
                   read_noise_stddev_input = 0.1,
                   device_variation_stddev_input = 0.0,
                   device_stuck_on_prob_input = 0.00,
                   device_stuck_off_prob_input = 0.00,
                   save_results_input = True)


0 Batch accuracy: 0.72 Test accuracy: 0.6556
1 Batch accuracy: 0.6 Test accuracy: 0.5914
2 Batch accuracy: 0.7 Test accuracy: 0.7075
3 Batch accuracy: 0.5 Test accuracy: 0.5555
4 Batch accuracy: 0.82 Test accuracy: 0.7896
5 Batch accuracy: 0.76 Test accuracy: 0.7968
6 Batch accuracy: 0.8 Test accuracy: 0.776
7 Batch accuracy: 0.82 Test accuracy: 0.7581
8 Batch accuracy: 0.9 Test accuracy: 0.8142
9 Batch accuracy: 0.6 Test accuracy: 0.6915
10 Batch accuracy: 0.78 Test accuracy: 0.8267
11 Batch accuracy: 0.9 Test accuracy: 0.8334
12 Batch accuracy: 0.84 Test accuracy: 0.8398
13 Batch accuracy: 0.82 Test accuracy: 0.8183
14 Batch accuracy: 0.8 Test accuracy: 0.843
15 Batch accuracy: 0.92 Test accuracy: 0.846
16 Batch accuracy: 0.78 Test accuracy: 0.826
17 Batch accuracy: 0.82 Test accuracy: 0.8299
18 Batch accuracy: 0.84 Test accuracy: 0.8476
19 Batch accuracy: 0.86 Test accuracy: 0.8514
20 Batch accuracy: 0.84 Test accuracy: 0.8119
21 Batch accuracy: 0.88 Test accuracy: 0.829
22 Batch ac

In [ ]:
# mock simulation using np.arange for device states
run_MLP_simulation(num_epochs_input = 100, 
                   hardware_simulation_input = False, 
                   device_states_input = False,
                   read_noise_mean_input = 0.0,
                   read_noise_stddev_input = 0.0,
                   device_variation_stddev_input = 0.0,
                   device_stuck_on_prob_input = 0.0,
                   device_stuck_off_prob_input = 0.0,
                   save_results_input = True)

0 Batch accuracy: 0.88 Test accuracy: 0.8849
1 Batch accuracy: 0.86 Test accuracy: 0.9082
2 Batch accuracy: 0.92 Test accuracy: 0.9206
3 Batch accuracy: 0.96 Test accuracy: 0.9269
4 Batch accuracy: 1.0 Test accuracy: 0.9298
5 Batch accuracy: 0.98 Test accuracy: 0.9327
6 Batch accuracy: 0.96 Test accuracy: 0.9368
7 Batch accuracy: 0.96 Test accuracy: 0.9404
8 Batch accuracy: 0.96 Test accuracy: 0.944
9 Batch accuracy: 0.98 Test accuracy: 0.9456
10 Batch accuracy: 0.98 Test accuracy: 0.9467
11 Batch accuracy: 0.9 Test accuracy: 0.9497
12 Batch accuracy: 0.92 Test accuracy: 0.9499
13 Batch accuracy: 0.98 Test accuracy: 0.9533
14 Batch accuracy: 0.94 Test accuracy: 0.9533
15 Batch accuracy: 0.94 Test accuracy: 0.9553
16 Batch accuracy: 1.0 Test accuracy: 0.9558
17 Batch accuracy: 0.94 Test accuracy: 0.9566
18 Batch accuracy: 0.98 Test accuracy: 0.9581
19 Batch accuracy: 0.96 Test accuracy: 0.9574
20 Batch accuracy: 0.96 Test accuracy: 0.9579
21 Batch accuracy: 0.98 Test accuracy: 0.9595
22

# Results

## software only (~2 mins , 10 sec)
- Learning rate = 0.01
0 Batch accuracy: 0.88 Test accuracy: 0.8849
1 Batch accuracy: 0.86 Test accuracy: 0.9082
2 Batch accuracy: 0.92 Test accuracy: 0.9206
3 Batch accuracy: 0.96 Test accuracy: 0.9269
4 Batch accuracy: 1.0 Test accuracy: 0.9298
5 Batch accuracy: 0.98 Test accuracy: 0.9327
6 Batch accuracy: 0.96 Test accuracy: 0.9368
7 Batch accuracy: 0.96 Test accuracy: 0.9404
8 Batch accuracy: 0.96 Test accuracy: 0.944
9 Batch accuracy: 0.98 Test accuracy: 0.9456
10 Batch accuracy: 0.98 Test accuracy: 0.9467
11 Batch accuracy: 0.9 Test accuracy: 0.9497
12 Batch accuracy: 0.92 Test accuracy: 0.9499
13 Batch accuracy: 0.98 Test accuracy: 0.9533
14 Batch accuracy: 0.94 Test accuracy: 0.9533
15 Batch accuracy: 0.94 Test accuracy: 0.9553
16 Batch accuracy: 1.0 Test accuracy: 0.9558
17 Batch accuracy: 0.94 Test accuracy: 0.9566
18 Batch accuracy: 0.98 Test accuracy: 0.9581
19 Batch accuracy: 0.96 Test accuracy: 0.9574
20 Batch accuracy: 0.96 Test accuracy: 0.9579
21 Batch accuracy: 0.98 Test accuracy: 0.9595
22 Batch accuracy: 0.94 Test accuracy: 0.9607
23 Batch accuracy: 0.98 Test accuracy: 0.9607
24 Batch accuracy: 0.98 Test accuracy: 0.961
25 Batch accuracy: 0.98 Test accuracy: 0.9612
26 Batch accuracy: 1.0 Test accuracy: 0.9623
27 Batch accuracy: 1.0 Test accuracy: 0.9628
28 Batch accuracy: 1.0 Test accuracy: 0.9622
29 Batch accuracy: 0.98 Test accuracy: 0.9632
30 Batch accuracy: 0.98 Test accuracy: 0.9636
31 Batch accuracy: 1.0 Test accuracy: 0.9625
32 Batch accuracy: 0.92 Test accuracy: 0.9637
33 Batch accuracy: 0.96 Test accuracy: 0.9652
34 Batch accuracy: 0.96 Test accuracy: 0.9635
35 Batch accuracy: 0.96 Test accuracy: 0.9637
36 Batch accuracy: 0.98 Test accuracy: 0.9655
37 Batch accuracy: 1.0 Test accuracy: 0.9656
38 Batch accuracy: 1.0 Test accuracy: 0.9662
39 Batch accuracy: 0.98 Test accuracy: 0.9666
40 Batch accuracy: 1.0 Test accuracy: 0.966
41 Batch accuracy: 0.98 Test accuracy: 0.9665
42 Batch accuracy: 0.96 Test accuracy: 0.9666
43 Batch accuracy: 0.98 Test accuracy: 0.9681
44 Batch accuracy: 1.0 Test accuracy: 0.9677
45 Batch accuracy: 0.98 Test accuracy: 0.9666
46 Batch accuracy: 0.96 Test accuracy: 0.9675
47 Batch accuracy: 1.0 Test accuracy: 0.9675
48 Batch accuracy: 1.0 Test accuracy: 0.9682
49 Batch accuracy: 1.0 Test accuracy: 0.9683
--- 140.81 seconds ---

## software only (~30 secs)
- Learning rate = 0.1
0 Batch accuracy: 0.94 Test accuracy: 0.9427
1 Batch accuracy: 0.98 Test accuracy: 0.9556
2 Batch accuracy: 0.98 Test accuracy: 0.9635
3 Batch accuracy: 1.0 Test accuracy: 0.9665
4 Batch accuracy: 1.0 Test accuracy: 0.965
5 Batch accuracy: 1.0 Test accuracy: 0.9678
6 Batch accuracy: 1.0 Test accuracy: 0.9699
7 Batch accuracy: 1.0 Test accuracy: 0.9722
8 Batch accuracy: 1.0 Test accuracy: 0.9727
9 Batch accuracy: 1.0 Test accuracy: 0.9718
10 Batch accuracy: 1.0 Test accuracy: 0.9746
11 Batch accuracy: 0.98 Test accuracy: 0.9745
12 Batch accuracy: 1.0 Test accuracy: 0.9765
13 Batch accuracy: 1.0 Test accuracy: 0.9765
14 Batch accuracy: 1.0 Test accuracy: 0.9762
15 Batch accuracy: 1.0 Test accuracy: 0.9753
16 Batch accuracy: 1.0 Test accuracy: 0.9754
17 Batch accuracy: 1.0 Test accuracy: 0.977
18 Batch accuracy: 1.0 Test accuracy: 0.9756
19 Batch accuracy: 1.0 Test accuracy: 0.9768
20 Batch accuracy: 1.0 Test accuracy: 0.977

## hardware ON, but just discrete states (20) (~2 min, 13 s)
0 Batch accuracy: 0.92 Test accuracy: 0.933
1 Batch accuracy: 0.92 Test accuracy: 0.938
2 Batch accuracy: 0.94 Test accuracy: 0.9418
3 Batch accuracy: 0.98 Test accuracy: 0.9431
4 Batch accuracy: 0.94 Test accuracy: 0.942
5 Batch accuracy: 0.96 Test accuracy: 0.9401
6 Batch accuracy: 0.92 Test accuracy: 0.9405
7 Batch accuracy: 0.96 Test accuracy: 0.9417
8 Batch accuracy: 0.92 Test accuracy: 0.9422
9 Batch accuracy: 0.96 Test accuracy: 0.9402
10 Batch accuracy: 1.0 Test accuracy: 0.9405
11 Batch accuracy: 0.92 Test accuracy: 0.9405
12 Batch accuracy: 0.96 Test accuracy: 0.9416
13 Batch accuracy: 0.94 Test accuracy: 0.9415
14 Batch accuracy: 0.98 Test accuracy: 0.9417
15 Batch accuracy: 0.98 Test accuracy: 0.9415
16 Batch accuracy: 0.96 Test accuracy: 0.9422
17 Batch accuracy: 0.96 Test accuracy: 0.9422
18 Batch accuracy: 0.88 Test accuracy: 0.9425
19 Batch accuracy: 0.98 Test accuracy: 0.942

## hardware ON, but just discrete states (100) ( min,  s)
0 Batch accuracy: 0.94 Test accuracy: 0.941
1 Batch accuracy: 0.96 Test accuracy: 0.9539
2 Batch accuracy: 0.98 Test accuracy: 0.9604
3 Batch accuracy: 1.0 Test accuracy: 0.9651
4 Batch accuracy: 1.0 Test accuracy: 0.9664
5 Batch accuracy: 1.0 Test accuracy: 0.9679
6 Batch accuracy: 0.98 Test accuracy: 0.9671
7 Batch accuracy: 1.0 Test accuracy: 0.971
8 Batch accuracy: 0.98 Test accuracy: 0.9705
9 Batch accuracy: 1.0 Test accuracy: 0.9709
10 Batch accuracy: 1.0 Test accuracy: 0.9711
11 Batch accuracy: 0.98 Test accuracy: 0.971
12 Batch accuracy: 1.0 Test accuracy: 0.9704
13 Batch accuracy: 0.98 Test accuracy: 0.9706
14 Batch accuracy: 1.0 Test accuracy: 0.9729
15 Batch accuracy: 1.0 Test accuracy: 0.9724
16 Batch accuracy: 1.0 Test accuracy: 0.9736

## hardware ON with... (1 min 11s)
- 20 states
- readnoise = 0.0 +/- 0.2
- stuckon = stuckoff = 0.05
- device_var = 0.1
0 Batch accuracy: 0.4 Test accuracy: 0.3264
1 Batch accuracy: 0.44 Test accuracy: 0.4225
2 Batch accuracy: 0.52 Test accuracy: 0.4578
3 Batch accuracy: 0.52 Test accuracy: 0.4802
4 Batch accuracy: 0.64 Test accuracy: 0.6497
5 Batch accuracy: 0.58 Test accuracy: 0.6118
6 Batch accuracy: 0.46 Test accuracy: 0.551
7 Batch accuracy: 0.52 Test accuracy: 0.5524
--- 71.81 seconds ---

## hardware ON with... (1 min 3s)
- 20 states
- readnoise = 0.0 +/- 0.05
- stuckon = stuckoff = 0.05
- device_var = 0.1
0 Batch accuracy: 0.88 Test accuracy: 0.8143
1 Batch accuracy: 0.94 Test accuracy: 0.8872
2 Batch accuracy: 0.92 Test accuracy: 0.8669
3 Batch accuracy: 0.78 Test accuracy: 0.8651
4 Batch accuracy: 0.96 Test accuracy: 0.8995
5 Batch accuracy: 0.9 Test accuracy: 0.9091
6 Batch accuracy: 0.92 Test accuracy: 0.8873
7 Batch accuracy: 0.88 Test accuracy: 0.8694
--- 63.78 seconds ---


## hardware ON with... (17 mins)
- 100 states
- readnoise = 0.0 +/- 0.05
- stuckon = stuckoff = 0.05
- device_var = 0.1
0 Batch accuracy: 0.86 Test accuracy: 0.8376
1 Batch accuracy: 0.96 Test accuracy: 0.8902
2 Batch accuracy: 0.94 Test accuracy: 0.8869
3 Batch accuracy: 0.9 Test accuracy: 0.8921
4 Batch accuracy: 0.94 Test accuracy: 0.9196
5 Batch accuracy: 0.9 Test accuracy: 0.9166
6 Batch accuracy: 0.94 Test accuracy: 0.9174
7 Batch accuracy: 0.88 Test accuracy: 0.9134
8 Batch accuracy: 0.88 Test accuracy: 0.884
9 Batch accuracy: 0.88 Test accuracy: 0.9095
10 Batch accuracy: 0.92 Test accuracy: 0.913
11 Batch accuracy: 0.86 Test accuracy: 0.909
12 Batch accuracy: 0.94 Test accuracy: 0.9265
13 Batch accuracy: 0.9 Test accuracy: 0.901
14 Batch accuracy: 0.92 Test accuracy: 0.9157
15 Batch accuracy: 0.92 Test accuracy: 0.8996
16 Batch accuracy: 0.9 Test accuracy: 0.9165
17 Batch accuracy: 0.92 Test accuracy: 0.917
18 Batch accuracy: 0.96 Test accuracy: 0.926
19 Batch accuracy: 0.94 Test accuracy: 0.9229
20 Batch accuracy: 0.98 Test accuracy: 0.9061
21 Batch accuracy: 0.94 Test accuracy: 0.9252
22 Batch accuracy: 0.92 Test accuracy: 0.9188
23 Batch accuracy: 0.96 Test accuracy: 0.9229
24 Batch accuracy: 0.94 Test accuracy: 0.9326
25 Batch accuracy: 0.96 Test accuracy: 0.9163
26 Batch accuracy: 0.94 Test accuracy: 0.923
27 Batch accuracy: 0.92 Test accuracy: 0.9222
28 Batch accuracy: 0.94 Test accuracy: 0.926
29 Batch accuracy: 0.98 Test accuracy: 0.9132
30 Batch accuracy: 0.96 Test accuracy: 0.9141
31 Batch accuracy: 0.94 Test accuracy: 0.9229
32 Batch accuracy: 0.88 Test accuracy: 0.9223
33 Batch accuracy: 0.9 Test accuracy: 0.8704
34 Batch accuracy: 0.9 Test accuracy: 0.9216
35 Batch accuracy: 0.92 Test accuracy: 0.9069
36 Batch accuracy: 0.94 Test accuracy: 0.9095
37 Batch accuracy: 0.92 Test accuracy: 0.9306
38 Batch accuracy: 0.9 Test accuracy: 0.9147
39 Batch accuracy: 0.94 Test accuracy: 0.9339
40 Batch accuracy: 0.96 Test accuracy: 0.9148
41 Batch accuracy: 0.96 Test accuracy: 0.8807
42 Batch accuracy: 0.92 Test accuracy: 0.8752
43 Batch accuracy: 0.92 Test accuracy: 0.9108
44 Batch accuracy: 0.92 Test accuracy: 0.9234
45 Batch accuracy: 0.94 Test accuracy: 0.9276
46 Batch accuracy: 0.94 Test accuracy: 0.9113
47 Batch accuracy: 0.9 Test accuracy: 0.914
48 Batch accuracy: 0.98 Test accuracy: 0.9277
49 Batch accuracy: 0.9 Test accuracy: 0.9335
--- 995.70 seconds ---

## hardware ON with... (14 mins)
- 100 states
- readnoise = 0.0 +/- 0.00
- stuckon = stuckoff = 0.05
- device_var = 0.1

0 Batch accuracy: 0.92 Test accuracy: 0.9238
1 Batch accuracy: 0.92 Test accuracy: 0.9367
2 Batch accuracy: 0.96 Test accuracy: 0.9368
3 Batch accuracy: 0.96 Test accuracy: 0.9459
4 Batch accuracy: 0.98 Test accuracy: 0.9495
5 Batch accuracy: 0.98 Test accuracy: 0.9565
6 Batch accuracy: 0.9 Test accuracy: 0.9342
7 Batch accuracy: 0.98 Test accuracy: 0.9486
8 Batch accuracy: 0.96 Test accuracy: 0.9432
9 Batch accuracy: 0.96 Test accuracy: 0.948
10 Batch accuracy: 1.0 Test accuracy: 0.9513
11 Batch accuracy: 0.94 Test accuracy: 0.9572
12 Batch accuracy: 1.0 Test accuracy: 0.9436
13 Batch accuracy: 0.92 Test accuracy: 0.9293
14 Batch accuracy: 0.96 Test accuracy: 0.9586
15 Batch accuracy: 1.0 Test accuracy: 0.9583
16 Batch accuracy: 0.98 Test accuracy: 0.9644
17 Batch accuracy: 1.0 Test accuracy: 0.9469
18 Batch accuracy: 0.9 Test accuracy: 0.9589
19 Batch accuracy: 0.98 Test accuracy: 0.9617
20 Batch accuracy: 0.96 Test accuracy: 0.9519
21 Batch accuracy: 0.94 Test accuracy: 0.9536
22 Batch accuracy: 0.96 Test accuracy: 0.9654
23 Batch accuracy: 0.98 Test accuracy: 0.9594
24 Batch accuracy: 0.98 Test accuracy: 0.9655
25 Batch accuracy: 1.0 Test accuracy: 0.9624
26 Batch accuracy: 0.96 Test accuracy: 0.9626
27 Batch accuracy: 1.0 Test accuracy: 0.958
28 Batch accuracy: 1.0 Test accuracy: 0.9532
29 Batch accuracy: 1.0 Test accuracy: 0.9704
30 Batch accuracy: 0.98 Test accuracy: 0.9576
31 Batch accuracy: 1.0 Test accuracy: 0.9594
32 Batch accuracy: 0.98 Test accuracy: 0.9676
33 Batch accuracy: 1.0 Test accuracy: 0.9568
34 Batch accuracy: 1.0 Test accuracy: 0.97
35 Batch accuracy: 0.98 Test accuracy: 0.9625
36 Batch accuracy: 1.0 Test accuracy: 0.9638
37 Batch accuracy: 0.98 Test accuracy: 0.9559
38 Batch accuracy: 1.0 Test accuracy: 0.9564
39 Batch accuracy: 0.98 Test accuracy: 0.9416
40 Batch accuracy: 0.98 Test accuracy: 0.9536
41 Batch accuracy: 0.98 Test accuracy: 0.9669
42 Batch accuracy: 0.96 Test accuracy: 0.9723
43 Batch accuracy: 0.94 Test accuracy: 0.9654
44 Batch accuracy: 0.96 Test accuracy: 0.9631
45 Batch accuracy: 0.98 Test accuracy: 0.9669
46 Batch accuracy: 1.0 Test accuracy: 0.9717
47 Batch accuracy: 0.94 Test accuracy: 0.9653
48 Batch accuracy: 0.92 Test accuracy: 0.9347
49 Batch accuracy: 1.0 Test accuracy: 0.9634
--- 875.17 seconds ---



In [ ]:
weights_before

array([-0.00363304, -0.16495076, -0.00245212, -0.07429077, -0.0231128 ,
       -0.34697148,  0.3885961 ,  0.30065742,  0.22381195, -0.0381159 ,
        0.00310417,  0.08535535,  0.17385383,  0.30208942,  0.125374  ,
        0.00897076,  0.07525111, -0.29270896, -0.18102363,  0.06648245,
       -0.01434219, -0.12452476, -0.23496754,  0.24518795, -0.03454113,
        0.09062955,  0.05114428, -0.01170656,  0.31281415,  0.05068856,
       -0.07658286, -0.3558821 , -0.02831533,  0.29215947, -0.01215394,
        0.05968314,  0.26844263,  0.34673175,  0.00499299,  0.01214231,
       -0.26232064, -0.18391348, -0.30076265,  0.08745792,  0.04777403,
       -0.30305094,  0.33885407, -0.22074407, -0.01648456,  0.14930668,
       -0.10613731, -0.15380175,  0.04764128,  0.1736799 , -0.05961963,
       -0.23478894, -0.18598206, -0.05220306, -0.03114807,  0.04526016,
       -0.13837756,  0.17210205, -0.11568228, -0.06414939, -0.07839669,
       -0.31229424, -0.2896966 ,  0.03144627, -0.03642305, -0.16

In [ ]:
weights_after

array([-0. , -0.2, -0. , -0.1, -0. , -0.3,  0.4,  0.3,  0.2, -0. , -0. ,
        0.1,  0.2,  0.3,  0.1, -0. ,  0.1, -0.3, -0.2,  0.1, -0. , -0.1,
       -0.2,  0.2, -0. ,  0.1,  0.1, -0. ,  0.3,  0.1, -0.1, -0.4, -0. ,
        0.3, -0. ,  0.1,  0.3,  0.3, -0. , -0. , -0.3, -0.2, -0.3,  0.1,
       -0. , -0.3,  0.3, -0.2, -0. ,  0.1, -0.1, -0.2, -0. ,  0.2, -0.1,
       -0.2, -0.2, -0.1, -0. , -0. , -0.1,  0.2, -0.1, -0.1, -0.1, -0.3,
       -0.3, -0. , -0. , -0.2, -0. ,  0.2, -0. , -0.1, -0. ,  0.2, -0. ,
        0.1,  0.4, -0.1,  0.2, -0.1, -0.1, -0. ,  0.1,  0.2,  0.1, -0.4,
       -0. , -0.1,  0.1, -0.2,  0.1, -0.2, -0.2, -0.1, -0.1, -0. ,  0.1,
       -0.2,  0.1, -0.1, -0. , -0.4,  0.1, -0.3, -0.1,  0.2,  0.1, -0.1,
        0.3,  0.1,  0.3,  0.1,  0.1,  0.2, -0.3,  0.2, -0.2, -0. ,  0.1,
        0.2,  0.1,  0.1, -0.3, -0.1,  0.3, -0. , -0.2, -0.2,  0.1,  0.2,
       -0. ,  0.3, -0.3,  0.1,  0.2, -0.1, -0.2, -0.3,  0.1,  0.1, -0.3,
       -0. , -0. ,  0.1, -0. ,  0.1,  0.1,  0.3,  0

In [ ]:
AlexNet = maxpool lots of layers
Vggnet = after alexnet, less intensive convolutions; higher num of params and lower accuracy
Resnet = 
inception = best but lots improve